<a href="https://colab.research.google.com/github/sunyeul/ToyProjectLab/blob/feature%2Fencrypt-api-key-sample/Encrypt_API_Key/encrypt_api_key.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade -qq cryptography

In [2]:
# generate_single_encryption_key.py

import os
import base64

def generate_key():
    key = base64.urlsafe_b64encode(os.urandom(32))
    return key

# 하나의 암호화 키 생성
single_key = generate_key()

with open("single_encryption_key.key", "wb") as key_file:
    key_file.write(single_key)

In [3]:
# encrypt_api_keys.py

from cryptography.fernet import Fernet
import json

def encrypt_api_key(api_key, key):
    f = Fernet(key)
    encrypted_api_key = f.encrypt(api_key.encode())
    return encrypted_api_key

# API 키 예시 (API 이름과 키 값)
api_keys = {
    "service1": "api_key_1",  # os.environ.get("api_key_1")
    "service2": "api_key_2",  # os.environ.get("api_key_2")
    "service3": "api_key_3"  # os.environ.get("api_key_3")
}

# API 키 및 암호화 키 저장을 위한 딕셔너리
encrypted_api_keys = {}

# 각 API 키를 암호화
for service, api_key in api_keys.items():
    encrypted_api_key = encrypt_api_key(api_key, single_key)
    encrypted_api_keys[service] = encrypted_api_key.decode()

# 암호화된 API 키 및 암호화 키를 JSON 파일로 저장
with open("encrypted_api_keys.json", "w") as f:
    json.dump(encrypted_api_keys, f)

In [4]:
# decrypt_api_keys.py

import json

def decrypt_api_key(encrypted_api_key, key):
    f = Fernet(key)
    decrypted_api_key = f.decrypt(encrypted_api_key).decode()
    return decrypted_api_key


# 암호화된 API 키를 JSON 파일에서 불러오기
with open("encrypted_api_keys.json", "r") as f:
    encrypted_api_keys = json.load(f)

# 암호화 키 불러오기
with open("single_encryption_key.key", "rb") as key_file:
    single_key = key_file.read()

# 모든 API 키를 복호화하고 출력
for service in encrypted_api_keys:
    encrypted_api_key = encrypted_api_keys[service].encode()

    decrypted_api_key = decrypt_api_key(encrypted_api_key, single_key)
    print(f"{service} Decrypted API key: {decrypted_api_key}")

service1 Decrypted API key: api_key_1
service2 Decrypted API key: api_key_2
service3 Decrypted API key: api_key_3
